In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("regdata.csv")

In [2]:
print(data.columns)
data = data.filter(['date2','hour','vader_sent_hourly mean','Daily Weight count','Close','price dir','movement'])
data

Index(['Unnamed: 0', 'Unnamed: 0.1', 'date2', 'hour', 'Daily Weight count',
       'blob_sent_hourly mean', 'vader_sent_hourly mean', 'day n hour',
       'tweet dir', 'vsent dir', 'bsent dir', 'bsent movement',
       'tweet movement', 'vsent movement', 'vesent movement', 'Close',
       '5hr MA', 'price dir', 'movement', 'date2.1', 'month', 'hour.1'],
      dtype='object')


,date2,hour,vader_sent_hourly mean,Daily Weight count,Close,price dir,movement
0,2019-01-01,0,0.147142,735,3700.31,0.000,0
1,2019-01-01,1,0.158979,603,3689.69,3.538,1
2,2019-01-01,2,0.151551,629,3690.00,3.656,1
3,2019-01-01,3,0.183226,546,3693.13,1.340,1
4,2019-01-01,4,0.202897,519,3692.71,-2.038,0
...,...,...,...,...,...,...,...
7146,2019-10-31,19,0.209381,1150,9211.04,-5.522,0
7147,2019-10-31,20,0.177808,1060,9185.98,-5.600,0
7148,2019-10-31,21,0.153912,960,9126.20,-22.674,0
7149,2019-10-31,22,0.138465,844,9122.69,-24.144,0


# Normalizing values

In [3]:
time_len = data.shape[0]
print(time_len)
x = data.filter(['vader_sent_hourly mean','Close','Daily Weight count',"movement"])

from sklearn import preprocessing

xx = x.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(xx)
#df = pd.DataFrame(x_scaled)
print(x_scaled)


7151
      vader_sent_hourly mean    Close  Daily Weight count  movement
0                   0.147142  3700.31                 735         0
1                   0.158979  3689.69                 603         1
2                   0.151551  3690.00                 629         1
3                   0.183226  3693.13                 546         1
4                   0.202897  3692.71                 519         0
...                      ...      ...                 ...       ...
7146                0.209381  9211.04                1150         0
7147                0.177808  9185.98                1060         0
7148                0.153912  9126.20                 960         0
7149                0.138465  9122.69                 844         0
7150                0.192858  9140.85                 734         0

[7151 rows x 4 columns]
[[0.30218862 0.0315168  0.19933278 0.        ]
 [0.32328959 0.03051438 0.16263553 1.        ]
 [0.31004717 0.03054364 0.16986378 1.        ]
 ...
 [0.3142

# Use series_to_supervised to get time series data

In [4]:

from sklearn.preprocessing import LabelEncoder

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
# input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (df.columns[j], i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (df.columns[j])) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (df.columns[j], i)) for j in range(n_vars)]
# put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 
# integer encode direction
encoder = LabelEncoder()
xx[:,3] = encoder.fit_transform(xx[:,3])
# ensure all data is float
xx = xx.astype('float32')
# normalize features
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(xx)
# frame as supervised learning
reframed = series_to_supervised(scaled, 2, 1)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[8,9,10]], axis=1, inplace=True)
print(reframed.head())


   var0(t-2)  var1(t-2)  var2(t-2)  var3(t-2)  var0(t-1)  var1(t-1)  \
2   0.302189   0.031517   0.199333        0.0   0.323290   0.030514   
3   0.323290   0.030514   0.162636        1.0   0.310047   0.030544   
4   0.310047   0.030544   0.169864        1.0   0.366514   0.030839   
5   0.366514   0.030839   0.146789        1.0   0.401580   0.030799   
6   0.401580   0.030799   0.139283        0.0   0.277331   0.031482   

   var2(t-1)  var3(t-1)  var3(t)  
2   0.162636        1.0      1.0  
3   0.169864        1.0      1.0  
4   0.146789        1.0      0.0  
5   0.139283        0.0      0.0  
6   0.142341        0.0      1.0  


# LSTM MODEL 


In [13]:
import keras
from keras.datasets import reuters, imdb
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, GRU, Dense, Dropout, Activation, Embedding, ConvLSTM2D,CuDNNLSTM,Bidirectional, Conv2D,Conv1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from keras import metrics as met2
import math
from numpy import concatenate

# train test split and reshaping

In [14]:
labels = reframed.values[:,-1]
#labels = labels.astype('int')
print(reframed.shape)
features = reframed.values[:,:8]
print(features.shape,labels.shape)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)
x_train = x_train.reshape((x_train.shape[0], 2, 4))
x_test = x_test.reshape((x_test.shape[0],2, 4))
print(x_train.shape)

(7149, 9)
(7149, 8) (7149,)
(5719, 2, 4)


# Compiling model

In [15]:

# design network
#for i in range(1,10):
model = Sequential()
#model.add(Conv2D(100, 2, activation='relu'))
#model.add(SimpleRNN(32))
model.add(Bidirectional(LSTM((50), input_shape=(x_train.shape[1], x_train.shape[2]))))
#model.add(ConvLSTM2D(2,kernel_size = (3,3),input_shape =(None,x_train.shape[1], x_train.shape[2],1) ,padding='same', return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss='binary_crossentropy', metrics=[met2.binary_accuracy], optimizer='adam')
# fit network
history = model.fit(x_train, y_train, epochs=50, batch_size=100, validation_data=(x_test, y_test), verbose=2, shuffle=False)
# plot history
print(model.summary())



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 5719 samples, validate on 1430 samples
Epoch 1/50
 - 1s - loss: 0.6542 - binary_accuracy: 0.6583 - val_loss: 0.6106 - val_binary_accuracy: 0.7329
Epoch 2/50
 - 0s - loss: 0.5844 - binary_accuracy: 0.7272 - val_loss: 0.5453 - val_binary_accuracy: 0.7580
Epoch 3/50
 - 0s - loss: 0.5546 - binary_accuracy: 0.7563 - val_loss: 0.5338 - val_binary_accuracy: 0.7755
Epoch 4/50
 - 0s - loss: 0.5493 - binary_accuracy: 0.7615 - val_loss: 0.5312 - val_binary_accuracy: 0.7755
Epoch 5/50
 - 0s - loss: 0.5486 - binary_accuracy: 0.7615 - val_loss: 0.5305 - val_binary_accuracy: 0.7755
Epoch 6/50
 - 0s - loss: 0.5482 - binary_accuracy: 0.7615 - val_loss: 0.5300 - val_binary_accuracy: 0.7755
Epoch 7/50
 - 0s - loss: 0.5471 - binary_accuracy: 0.7615 - val_loss: 0.5294 - val_binary_accuracy: 0.7755
Epoch 8/50
 - 0s - loss: 0.5470 - binary_accuracy: 0.7615 - val_loss: 0.5293 - val_binary_accuracy: 0.7755
E

In [16]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.show()

<Figure size 640x480 with 1 Axes>

# Evaluation

In [17]:
# make a prediction

#x_test = test_X.reshape((x_test.shape[0], x_test.shape[2]))
yhat = model.predict(x_test)
#print(yhat.score)
rmse = math.sqrt(mean_squared_error(y_test, yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.415


In [19]:

#print(yhat[0:90])
yhat = yhat.astype("float64")
y2 = yhat
for i in range(yhat.shape[0]):
    if y2[i] >0.5:
        y2[i] = 1
    else:
        y2[i] = 0
matrix = metrics.confusion_matrix(y_test,yhat)
#acc = met2.binary_accuracy(y_test,keras.backend.round(yhat),threshold=0.5)
print(matrix)
#print(acc)

score = model.evaluate(x_test, y_test, batch_size=72, verbose=1)
print('Test score:', score[1])


[[507 167]
 [154 602]]
1430/1430 [==============================] - 0s 11us/step
Test score: 0.7755244970321655
